# Preprocessing der Trainingsdaten

In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import MinMaxScaler

print("Current working directory: " + os.getcwd())
train_data_path = os.path.join(os.getcwd(), "resources\\train_data\\")
print("Train data path: " + train_data_path)

Current working directory: C:\Users\Valentin\Documents\10. Semester\WR2\Projekt 3 - FNN CLF\WR-2-3-FFN-CLF
Train data path: C:\Users\Valentin\Documents\10. Semester\WR2\Projekt 3 - FNN CLF\WR-2-3-FFN-CLF\resources\train_data\


### Datensatz 1

In [2]:
# Daten laden
data1_path = os.path.join(train_data_path, "2022-06-29-05-12-03-train_data_dump.csv")
print("data1 path: " + data1_path)
data_set1 = pd.read_csv(data1_path, header = None)
print(data_set1.shape)
param1_path = os.path.join(train_data_path, "2022-06-29-05-14-01-parameter_dump.csv")
print("param1 path: " + param1_path)
parameter_set1 = pd.read_csv(param1_path, names = ["coefficentType","parameterNames","parameterValues"])
print(parameter_set1.shape)
parameter_set1.head()

data1 path: C:\Users\Valentin\Documents\10. Semester\WR2\Projekt 3 - FNN CLF\WR-2-3-FFN-CLF\resources\train_data\2022-06-29-05-12-03-train_data_dump.csv
(13179, 4801)
param1 path: C:\Users\Valentin\Documents\10. Semester\WR2\Projekt 3 - FNN CLF\WR-2-3-FFN-CLF\resources\train_data\2022-06-29-05-14-01-parameter_dump.csv
(13179, 3)


,coefficentType,parameterNames,parameterValues
0,Constant,"affectedSubdomains,rhoMin,rhoMax,indexShiftx,i...","6,10,1,1000000,0,0,,,"
1,Constant,"affectedSubdomains,rhoMin,rhoMax,indexShiftx,i...","6,10,1,1000000,0,0,,,"
2,Constant,"affectedSubdomains,rhoMin,rhoMax,indexShiftx,i...","6,10,1,1000000,0,0,,,"
3,Constant,"affectedSubdomains,rhoMin,rhoMax,indexShiftx,i...","6,10,1,1000000,0,0,,,"
4,Constant,"affectedSubdomains,rhoMin,rhoMax,indexShiftx,i...","6,10,1000,1,0,0,,,"


In [3]:
data_set1.head()

,0,1,2,3,4,5,6,7,8,9,...,4791,4792,4793,4794,4795,4796,4797,4798,4799,4800
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Datensatz 2

In [4]:
# Daten laden
data2_path = os.path.join(train_data_path, "2022-06-29-02-31-52-train_data_dump.csv")
print("data2 path: " + data2_path)
data_set2 = pd.read_csv(data2_path, header = None)
print(data_set2.shape)
param2_path = os.path.join(train_data_path, "2022-06-29-02-32-41-parameter_dump.csv")
print("param2 path: " + param2_path)
parameter_set2 = pd.read_csv(param2_path, names = ["coefficentType","parameterNames","parameterValues"])
print(parameter_set2.shape)
parameter_set2.head()

data2 path: C:\Users\Valentin\Documents\10. Semester\WR2\Projekt 3 - FNN CLF\WR-2-3-FFN-CLF\resources\train_data\2022-06-29-02-31-52-train_data_dump.csv
(24008, 4801)
param2 path: C:\Users\Valentin\Documents\10. Semester\WR2\Projekt 3 - FNN CLF\WR-2-3-FFN-CLF\resources\train_data\2022-06-29-02-32-41-parameter_dump.csv
(24008, 3)


,coefficentType,parameterNames,parameterValues
0,Constant,"affectedSubdomains,rhoMin,rhoMax,indexShiftx,i...","6,10,1,1000000,0,0,,,"
1,Constant,"affectedSubdomains,rhoMin,rhoMax,indexShiftx,i...","6,10,1,1000000,0,0,,,"
2,Constant,"affectedSubdomains,rhoMin,rhoMax,indexShiftx,i...","6,10,1,1000000,0,0,,,"
3,Constant,"affectedSubdomains,rhoMin,rhoMax,indexShiftx,i...","6,10,1,1000000,0,0,,,"
4,Constant,"affectedSubdomains,rhoMin,rhoMax,indexShiftx,i...","6,10,1000000,1,0,0,,,"


In [5]:
data_set2.head()

,0,1,2,3,4,5,6,7,8,9,...,4791,4792,4793,4794,4795,4796,4797,4798,4799,4800
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0


### Datensatz 3

In [6]:
# Daten laden
data3_path = os.path.join(train_data_path, "2022-06-29-15-19-21-train_data_dump.csv")
print("data3 path: " + data3_path)
data_set3 = pd.read_csv(data3_path, header = None)
print(data_set3.shape)
param3_path = os.path.join(train_data_path, "2022-06-29-15-21-37-parameter_dump.csv")
print("param3 path: " + param3_path)
parameter_set3 = pd.read_csv(param3_path, names = ["coefficentType","parameterNames","parameterValues"])
print(parameter_set3.shape)
parameter_set3.head()

data3 path: C:\Users\Valentin\Documents\10. Semester\WR2\Projekt 3 - FNN CLF\WR-2-3-FFN-CLF\resources\train_data\2022-06-29-15-19-21-train_data_dump.csv
(24008, 4801)
param3 path: C:\Users\Valentin\Documents\10. Semester\WR2\Projekt 3 - FNN CLF\WR-2-3-FFN-CLF\resources\train_data\2022-06-29-15-21-37-parameter_dump.csv
(24008, 3)


,coefficentType,parameterNames,parameterValues
0,Constant,"affectedSubdomains,rhoMin,rhoMax,indexShiftx,i...","6,10,1,1000000,0,0,,,"
1,Constant,"affectedSubdomains,rhoMin,rhoMax,indexShiftx,i...","6,10,1,1000000,0,0,,,"
2,Constant,"affectedSubdomains,rhoMin,rhoMax,indexShiftx,i...","6,10,1,1000000,0,0,,,"
3,Constant,"affectedSubdomains,rhoMin,rhoMax,indexShiftx,i...","6,10,1,1000000,0,0,,,"
4,Constant,"affectedSubdomains,rhoMin,rhoMax,indexShiftx,i...","6,10,1000000,1,0,0,,,"


In [7]:
data_set3.head()

,0,1,2,3,4,5,6,7,8,9,...,4791,4792,4793,4794,4795,4796,4797,4798,4799,4800
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0


#### Datensatz 4 (Konstant, Strips und Random)

In [8]:
# Daten laden
data4_path = os.path.join(train_data_path, "2022-07-07-05-38-17-train_data_dump_const_strips_rand.csv")
print("data3 path: " + data4_path)
data_set4 = pd.read_csv(data4_path, header = None)
print(data_set4.shape)
param4_path = os.path.join(train_data_path, "2022-07-07-05-38-17-parameter_dump_const_strips_rand.csv")
print("param4 path: " + param4_path)
parameter_set4 = pd.read_csv(param4_path, names = ["coefficentType","parameterNames","parameterValues"])
print(parameter_set4.shape)
parameter_set4.head()

data3 path: C:\Users\Valentin\Documents\10. Semester\WR2\Projekt 3 - FNN CLF\WR-2-3-FFN-CLF\resources\train_data\2022-07-07-05-38-17-train_data_dump_const_strips_rand.csv
(15088, 4801)
param4 path: C:\Users\Valentin\Documents\10. Semester\WR2\Projekt 3 - FNN CLF\WR-2-3-FFN-CLF\resources\train_data\2022-07-07-05-38-17-parameter_dump_const_strips_rand.csv
(15088, 3)


,coefficentType,parameterNames,parameterValues
0,Constant,"affectedSubdomains,rhoMin,rhoMax,indexShiftx,i...","6,10,1,1000000,0,0"
1,Constant,"affectedSubdomains,rhoMin,rhoMax,indexShiftx,i...","6,10,1,1000000,0,0"
2,Constant,"affectedSubdomains,rhoMin,rhoMax,indexShiftx,i...","6,10,1,1000000,0,0"
3,Constant,"affectedSubdomains,rhoMin,rhoMax,indexShiftx,i...","6,10,1,1000000,0,0"
4,Constant,"affectedSubdomains,rhoMin,rhoMax,indexShiftx,i...","6,10,1000000,1,0,0"


In [9]:
data_set4.head()

,0,1,2,3,4,5,6,7,8,9,...,4791,4792,4793,4794,4795,4796,4797,4798,4799,4800
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Datensatz 5 (RandomBlocks)

In [10]:
# Daten laden
data5_path = os.path.join(train_data_path, "2022-07-07-10-04-48-train_data_dump_randBlocks.csv")
print("data5 path: " + data5_path)
data_set5 = pd.read_csv(data5_path, header = None)
print(data_set5.shape)
param5_path = os.path.join(train_data_path, "2022-07-07-10-04-48-parameter_dump_randBlocks.csv")
print("param5 path: " + param5_path)
parameter_set5 = pd.read_csv(param5_path, names = ["coefficentType","parameterNames","parameterValues"])
print(parameter_set5.shape)
parameter_set5.head()

data5 path: C:\Users\Valentin\Documents\10. Semester\WR2\Projekt 3 - FNN CLF\WR-2-3-FFN-CLF\resources\train_data\2022-07-07-10-04-48-train_data_dump_randBlocks.csv
(28000, 4801)
param5 path: C:\Users\Valentin\Documents\10. Semester\WR2\Projekt 3 - FNN CLF\WR-2-3-FFN-CLF\resources\train_data\2022-07-07-10-04-48-parameter_dump_randBlocks.csv
(28000, 3)


,coefficentType,parameterNames,parameterValues
0,Constant,"affectedSubdomains,rhoMin,rhoMax,indexShiftx,i...","6,10,1,1000000,0,0,,,"
1,Constant,"affectedSubdomains,rhoMin,rhoMax,indexShiftx,i...","6,10,1,1000000,0,0,,,"
2,Constant,"affectedSubdomains,rhoMin,rhoMax,indexShiftx,i...","6,10,1,1000000,0,0,,,"
3,Constant,"affectedSubdomains,rhoMin,rhoMax,indexShiftx,i...","6,10,1,1000000,0,0,,,"
4,Constant,"affectedSubdomains,rhoMin,rhoMax,indexShiftx,i...","6,10,1000000,1,0,0,,,"


In [11]:
data_set5.head()

,0,1,2,3,4,5,6,7,8,9,...,4791,4792,4793,4794,4795,4796,4797,4798,4799,4800
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Assembliere die verschiedenen Ergebnisse anhand der Koeffizientenfunktion

In [12]:
df_base = pd.concat([data_set1,
                     data_set2,
                     data_set3,
                     data_set4,
                     data_set5
                    ],axis = 0,ignore_index = True)
paramdf_base = pd.concat([parameter_set1,
                          parameter_set2,
                          parameter_set3,
                          parameter_set4,
                          parameter_set5
                         ],axis = 0,ignore_index = True)

print(df_base.shape)
print(paramdf_base.shape)
df_base.head()

(104283, 4801)
(104283, 3)


,0,1,2,3,4,5,6,7,8,9,...,4791,4792,4793,4794,4795,4796,4797,4798,4799,4800
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
del data_set1
del data_set2
del data_set3
del data_set4
del data_set5

In [14]:
# Zähle Koeffizientenfunktionen
value_counts_pre_drop = paramdf_base['coefficentType'].value_counts()
print(value_counts_pre_drop)

Random Blocks        43419
Completely Random    27240
Strip                18348
Blocks               15238
Constant                38
Name: coefficentType, dtype: int64


In [15]:
# Schmeiße die Duplikate
duplicate_rows = df_base.duplicated()
print("shape with duplicates: ")
print(df_base.shape)
df_base = df_base.loc[~duplicate_rows,]
paramdf_base = paramdf_base.loc[~duplicate_rows,]
print("shape without duplicates: ")
print(df_base.shape)
print("Percentage of datapoints lost: " + str((100*sum(duplicate_rows)/len(duplicate_rows))) + "\%")
value_counts_post_drop = paramdf_base['coefficentType'].value_counts()
print(value_counts_post_drop)

shape with duplicates: 
(104283, 4801)
shape without duplicates: 
(56630, 4801)
Percentage of datapoints lost: 45.69584687820642\%
Random Blocks        32019
Completely Random    18444
Blocks                4930
Strip                 1233
Constant                 4
Name: coefficentType, dtype: int64


In [16]:
# Anzahl verloren gegangener Trainingspunkten
print(value_counts_post_drop-value_counts_pre_drop)

Blocks              -10308
Completely Random    -8796
Constant               -34
Random Blocks       -11400
Strip               -17115
Name: coefficentType, dtype: int64


In [17]:
df_base = pd.concat([paramdf_base,df_base], axis=1)
df_base.head()

NameError: name 'df' is not defined

### Konstante Koeffizienten

In [18]:
# coeff_fun_names = ["Constant","Strip","Blocks","Random Blocks","Completely Random"]
df_constant = df_base.loc[df_base['coefficentType'] == "Constant",]
print(df_constant.shape)

(4, 4804)


In [19]:
df_constant.iloc[:,-1].describe()

count    4.0
mean     0.0
std      0.0
min      0.0
25%      0.0
50%      0.0
75%      0.0
max      0.0
Name: 4800, dtype: float64

### Streifen Koeffizientenfunktion

In [20]:
# coeff_fun_names = ["Constant","Strip","Blocks","Random Blocks","Completely Random"]
df_strip = df_base.loc[df_base['coefficentType'] == "Strip",]
print(df_strip.shape)

(1233, 4804)


In [21]:
df_strip.iloc[:,-1].describe()

count    1233.000000
mean        0.473642
std         0.499507
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: 4800, dtype: float64

### Block Koeffizientenfunktion

In [22]:
# coeff_fun_names = ["Constant","Strip","Blocks","Random Blocks","Completely Random"]
df_blocks = df_base.loc[df_base['coefficentType'] == "Blocks",]
print(df_blocks.shape)

(4930, 4804)


In [23]:
df_blocks.iloc[:,-1].describe()

count    4930.000000
mean        0.114807
std         0.318822
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: 4800, dtype: float64

### Random Block Koeffizientenfunktion

In [24]:
# coeff_fun_names = ["Constant","Strip","Blocks","Random Blocks","Completely Random"]
df_randBlocks = df_base.loc[paramdf_base['coefficentType'] == "Random Blocks",]
print(df_randBlocks.shape)

(32019, 4804)


In [25]:
df_randBlocks.iloc[:,-1].describe()

count    32019.000000
mean         0.084106
std          0.277552
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: 4800, dtype: float64

### Random Koeffizientenfunktion

In [26]:
# coeff_fun_names = ["Constant","Strip","Blocks","Random Blocks","Completely Random"]
df_random = df_base.loc[paramdf_base['coefficentType'] == "Completely Random",]
print(df_random.shape)

(18444, 4804)


In [27]:
df_random.iloc[:,-1].describe()

count    18444.000000
mean         0.381425
std          0.485750
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: 4800, dtype: float64

## Datensatz zusammensetzen

In [28]:
randomState = 0
shareConstant = 1
shareStrip = 1
shareBlocks = 0.8
shareRandBlocks = 0.7
shareRand = 0

In [32]:
df = pd.concat([df_constant.sample(round(df_constant.shape[0] * shareConstant),random_state = randomState),
                df_strip.sample(round(df_strip.shape[0] * shareStrip),random_state = randomState),
                df_blocks.sample(round(df_blocks.shape[0] * shareBlocks),random_state = randomState),
                df_randBlocks.sample(round(df_randBlocks.shape[0] * shareRandBlocks),random_state = randomState),
                df_random.sample(round(df_random.shape[0] * shareRand),random_state = randomState)
               ], axis = 0, ignore_index = True)

print(df.shape)
df.head()

(27594, 4804)


,coefficentType,parameterNames,parameterValues,0,1,2,3,4,5,6,...,4791,4792,4793,4794,4795,4796,4797,4798,4799,4800
0,Constant,"affectedSubdomains,rhoMin,rhoMax,indexShiftx,i...","6,10,1,1000000,0,0,,,",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Constant,"affectedSubdomains,rhoMin,rhoMax,indexShiftx,i...","6,10,1000000,1,0,0,,,",0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
2,Constant,"affectedSubdomains,rhoMin,rhoMax,indexShiftx,i...","6,10,1,1000000,0,0,,,",1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,0
3,Constant,"affectedSubdomains,rhoMin,rhoMax,indexShiftx,i...","6,10,1,1000000,0,0,,,",1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,Strip,"height,nStrips,rhoMin,rhoMax,indexShifty","1,5,1,1000000,16,,,,",0,1,1,1,1,0,0,...,0,0,0,1,1,1,1,0,0,1


## Duplikate entfernen

In [35]:
# Exportiere Trainingsdaten
df.iloc[:,3:].to_csv(os.path.join(train_data_path, "training_dataset_ver2.csv"),index = False, header = False)
# Exportiere zugehörige Parameter
df.iloc[:,0:3].to_csv(os.path.join(train_data_path, "training_dataset_ver2_parameters.csv"),index = False, header = True)